In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn 
import pandas as pd
import os, gc, sys, time
import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

## **2.1数据集titanic**

In [2]:
train_file = './data/titanic/titanic_train.csv'
eval_file = './data/titanic/titanic_eval.csv'

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

train_df.head(5)

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [3]:
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')

train_df.head(10)

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
5,male,2.0,3,1,21.0750,Third,unknown,Southampton,n
6,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
7,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
8,female,4.0,1,1,16.7000,Third,G,Southampton,n
9,male,20.0,0,0,8.0500,Third,unknown,Southampton,y


## **2.2 feature_column的使用**
**离散特征**: one-hot编码     
**连续特征**: 分桶

In [4]:
categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class',
                       'deck', 'embark_town', 'alone']
numeric_columns = ['age', 'fare']

feature_columns = []

# 将类别特征转换为one-hot的特征
for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique()
    # 使用tf.feature_columns下的api
    feature_columns.append(tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(categorical_column, vocab)))
for numeric_column in numeric_columns:
    feature_columns.append(tf.feature_column.numeric_column(numeric_column, dtype=tf.float32))

**交叉特征**

In [6]:
feature_columns.append(tf.feature_column.indicator_column(tf.feature_column.crossed_column(['age', 'sex'],
                                 hash_bucket_size=100 # hash_bucket_size用来控制cross后特征变得很多
                                                                                          )))

In [7]:
def make_dataset(data_df, label_df, epochs = 10, shuffle = True, batch_size = 32):
    dataset = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
        dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

## **1.3  tf自带estimator**

有两种训练的模式
- 直接调用fit的方法
- 将model转换为estimator，然后再进行训练

In [8]:
output_dir = 'linear_model_cf'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

linear_estimator = tf.estimator.LinearClassifier(model_dir=output_dir, 
                                                     n_classes = 2,
                                                   feature_columns=feature_columns)
linear_estimator.train(input_fn = lambda : make_dataset(train_df, y_train, epochs=100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'linear_model_cf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FBB318CF98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras pass *_cons

In [9]:
linear_estimator.evaluate(input_fn = lambda : make_dataset(
    eval_df, y_eval, epochs = 1, shuffle = False))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-25T14:24:38Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from linear_model_cf\model.ckpt-1960
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-25-14:24:39
INFO:tensorflow:Saving dict for global step 1960: accuracy = 0.8030303, accuracy_baseline = 0.625, auc = 0.8584634, auc_pr

{'accuracy': 0.8030303,
 'accuracy_baseline': 0.625,
 'auc': 0.8584634,
 'auc_precision_recall': 0.7755953,
 'average_loss': 0.4578519,
 'label/mean': 0.375,
 'loss': 0.44055963,
 'precision': 0.7326733,
 'prediction/mean': 0.37647587,
 'recall': 0.74747473,
 'global_step': 1960}

In [10]:
output_dir = 'dnn_model_cf'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

dnn_estimator = tf.estimator.DNNClassifier(model_dir=output_dir, 
                                           n_classes = 2,
                                           hidden_units=[128, 128],
                                           activation_fn=tf.nn.relu,
                                           optimizer='Adam',
                                           feature_columns=feature_columns)
dnn_estimator.train(input_fn = lambda : make_dataset(train_df, y_train, epochs=100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'dnn_model_cf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FBB46C1898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to ru

In [11]:
dnn_estimator.evaluate(input_fn = lambda : make_dataset(
    eval_df, y_eval, epochs = 1, shuffle = False))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-25T14:25:05Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from dnn_model_cf\model.ckpt-1960
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-25-14:25:06
INFO:tensorflow:Saving dict for global step 1960: accuracy = 0.78409094, accuracy_baseline = 0.625, auc = 0.8408938, auc_prec

{'accuracy': 0.78409094,
 'accuracy_baseline': 0.625,
 'auc': 0.8408938,
 'auc_precision_recall': 0.76810956,
 'average_loss': 0.75915486,
 'label/mean': 0.375,
 'loss': 0.7029121,
 'precision': 0.6944444,
 'prediction/mean': 0.4201552,
 'recall': 0.75757575,
 'global_step': 1960}

estimator在model之外额外完成了很多操作，或保存tensorboard和model